# VencoPy Tutorial 4

This tutorial aims to give a more in depth overview into the GridModeler class and showcases some features that can be customised.

In [ ]:
import sys
import os
from os import path
import pandas as pd
from pathlib import Path
import pathlib
#from ruamel.yaml import YAML

sys.path.append(path.dirname(path.dirname(path.dirname(Path.cwd()))))

from vencopy.classes.dataParsers import DataParser
from vencopy.classes.tripDiaryBuilders import TripDiaryBuilder
from vencopy.classes.gridModelers import GridModeler
from vencopy.classes.flexEstimators import FlexEstimator
from vencopy.classes.evaluators import Evaluator
from vencopy.scripts.globalFunctions import loadConfigDict, createOutputFolders

print("Current working directory: {0}".format(os.getcwd()))

In [ ]:
configNames = ('globalConfig', 'localPathConfig', 'parseConfig', 'tripConfig', 'gridConfig', 'flexConfig', 'evaluatorConfig')
basePath = Path.cwd().parent.parent
configDict = loadConfigDict(configNames, basePath)
createOutputFolders(configDict=configDict)

# Adapt relative paths in config for tutorials
configDict['globalConfig']['pathRelative']['plots'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['plots']
configDict['globalConfig']['pathRelative']['parseOutput'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['parseOutput']
configDict['globalConfig']['pathRelative']['diaryOutput'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['diaryOutput']
configDict['globalConfig']['pathRelative']['gridOutput'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['gridOutput']
configDict['globalConfig']['pathRelative']['flexOutput'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['flexOutput']
configDict['globalConfig']['pathRelative']['evalOutput'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['evalOutput']

# Set reference dataset
datasetID = 'MiD17'

# Modify the localPathConfig file to point to the .csv file in the sampling folder in the tutorials directory where the dataset for the tutorials lies.
configDict['localPathConfig']['pathAbsolute'][datasetID] = pathlib.Path.cwd().parent / 'data_sampling'

# Similarly we modify the datasetID in the global config file
configDict['globalConfig']['files'][datasetID]['tripsDataRaw'] = datasetID + '.csv'

# We also modify the parseConfig by removing some of the columns that are normally parsed from the MiD, which are not available in our semplified test dataframe
del configDict['parseConfig']['dataVariables']['hhID']
del configDict['parseConfig']['dataVariables']['personID']


## GridModeler config file

Let's print the GridModeler config file.

In [ ]:
configDict['gridConfig']

As we can see the GridModeler config file contains two keys: chargingInfrastructureMappings and chargingInfrastructureDistributions. The first one basically sets for which trip purpose the infrastructure availability should be considered, the second one specifies the probabilities given for each location (trip purpose) and the respective charging power.

## _GridModeler_ class

The charging infrastructure allocation makes use of a basic charging infrastructure model, which assumes the availability of charging stations when vehicles are parked. Since the analytical focus of the framework lies on a regional level (NUTS1-NUTS0), the infrastructure model is kept simple in the current version.

Charging availability is allocated based on a binary True–False mapping to a respective trip purpose in the VencoPy-config. Thus, different scenarios describing different charging availability scenarios, e.g., at home or at home and at work etc. can be distinguished, but neither a regional differentiation nor a charging availability probability or distribution are assumed.

At the end of the execution of the GridModeler class, a given parking purpose diary parkingType(v,t) is transferred into a binary grid connection diary connectgrid(v,t) with the same format but consisting only of True–False values.

In [ ]:
# Run the first two classes to generate data
vpData = DataParser(datasetID=datasetID, configDict=configDict, loadEncrypted=False)
vpData.process()
vpTripDiary = TripDiaryBuilder(datasetID=datasetID, configDict=configDict, ParseData=vpData, debug=True)


In [ ]:
vpGrid = GridModeler(datasetID=datasetID, configDict=configDict)
vpGrid.assignSimpleGridViaPurposes()
vpGrid.writeOutGridAvailability()

In [ ]:
vpFlex = FlexEstimator(configDict=configDict, datasetID=datasetID, ParseData=vpData)
vpFlex.baseProfileCalculation()
vpFlex.filter()
vpFlex.aggregate()
vpFlex.correct()
vpFlex.normalize()
vpFlex.writeOut()


In [ ]:
vpEval = Evaluator(configDict=configDict, parseData=pd.Series(data=vpData, index=[datasetID]))
vpEval.plotProfiles(flexEstimator=vpFlex)

We can now change the grid availability from home to the workplace only and see how this affects the available charging flexibility.

In [ ]:
configDict['gridConfig']['chargingInfrastructureMappings']['HOME'] = False
configDict['gridConfig']['chargingInfrastructureMappings']['WORK'] = True
configDict['gridConfig']['chargingInfrastructureMappings']

In [ ]:
vpGrid = GridModeler(datasetID=datasetID, configDict=configDict)
vpGrid.assignSimpleGridViaPurposes()
vpGrid.writeOutGridAvailability()

In [ ]:
vpEval.plotProfiles(flexEstimator=vpFlex)

From the plots we can see how in the case of home charging, the grid connenction share varies between 0.4 and 1 for the fleet and it is 'homogeneously' distributed throughout the week, whereas in the workplace charging scenario it is limited to working days. 

## Next Steps

In the next tutorial, you will learn more in detail the internal workings of the FlexEstimator class and how to customise some settings.